In [8]:
import pandas as pd
import ast
from collections import Counter

# Read the csv files

import pandas as pd
class_file= pd.read_csv( r"Lexicon-Priority.csv" )
class_file.head()


import pandas as pd
final = pd.read_csv( r"After _ preprocessing.csv" )

# Define a function to get the word count as a list of tuples
def get_word_count(row):
    # Convert the string representation of the list into an actual list
    words = ast.literal_eval(row)
    # Count the occurrences of each word
    word_count = Counter(words)
    return [(word, count) for word, count in word_count.items()]

# Apply the function to the 'data' column
final["most_frequent_words"] = final["stemmed"].apply(get_word_count)

# Create a dictionary to store the word labels
word_labels = {}

# Iterate through the class_file DataFrame
for index, row in class_file.iterrows():
    # Iterate through the columns in the row
    for column in class_file.columns:
        # If the column value is not NaN
        if not pd.isna(row[column]):
            # Add the column value (label) to the dictionary for each word
            for word in row[column].split(","):
                word_labels[word.strip()] = column

# Create a new column 'final_label' in final DataFrame
final["final_label"] = ""

# Iterate through the final DataFrame
for index, row in final.iterrows():
    word_count = {}
    # Iterate through the most_frequent_words column
    for word, count in row["most_frequent_words"]:
        if word in word_labels:
            label = word_labels[word]
            if label in word_count:
                word_count[label] += count
            else:
                word_count[label] = count
    if word_count:
        final.at[index, "final_label"] = max(word_count, key=word_count.get)
    else:
        final.at[index, "final_label"] = "No match found"

final.to_csv( r"Automatic Label.csv", index=False)